In [1]:
import re
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1671601772602_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
config_raw = spark.sparkContext.textFile('s3://userconfig-6998/userConfigs.txt')
hardware_raw = spark.sparkContext.textFile('s3://userconfig-6998/hardware.txt')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
config = config_raw.map(lambda x: x.split(','))
hardware = hardware_raw.map(lambda x: x.split(',')).map(lambda x: (x[0], x[1], float(x[2])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# config.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['testuser0', '1671592780116', 'student', 'Intel Core i5-12400', 'MSI MSI GTX 1070 Ti 8GB Gaming', 'Crucial Ballistix Sport DDR4 2400 C16 8x4GB', 'WD Black SN850 NVMe PCIe M.2 2TB', 'WD VelociRaptor 1TB']]

In [5]:
# hardware.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Toshiba HDWD105 500GB', 'HDD', 59.99)]

In [6]:
hardwareDF = hardware.toDF(["name", "category", "price"])
# hardwareDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)

In [7]:
configDF = config.toDF(["user", "config_id", "category", "CPU", "GPU", "RAM", "SSD", "HDD"])
# configDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user: string (nullable = true)
 |-- config_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- CPU: string (nullable = true)
 |-- GPU: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- SSD: string (nullable = true)
 |-- HDD: string (nullable = true)

In [9]:
configDF.createOrReplaceTempView("config")
hardwareDF.createOrReplaceTempView("hardware")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
config_joined = spark.sql("select config.category, h1.price, h2.price, h3.price, h4.price, h5.price from config, hardware h1, hardware h2, hardware h3, hardware h4, hardware h5 where config.CPU = h1.name AND config.GPU = h2.name AND config.RAM = h3.name AND config.SSD = h4.name AND config.HDD = h5.name")
# config_joined.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(category='student', price=234.99, price=189.99, price=47.25, price=79.99, price=38.81)]

In [18]:
config_reduced = config_joined.rdd.map(lambda x: (x[0], [x[1], x[2], x[3], x[4], x[5]])).reduceByKey(lambda x, y: [x[0]+y[0], x[1]+y[1], x[2]+y[2], x[3]+y[3], x[4]+y[4]])
# config_reduced.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('common_user', [248.99, 49.99, 20.98, 109.99, 49.99]), ('gamer', [234.99, 469.99, 47.25, 109.99, 134.99]), ('student', [1996.43, 1847.0800000000002, 331.24, 877.96, 719.76])]

In [19]:
ratio = config_reduced.map(lambda x: (x[0], [x[1][0], x[1][1], x[1][2], x[1][3], x[1][4], x[1][0] + x[1][1] + x[1][2] + x[1][3] + x[1][4]])).map(lambda x: (x[0], [x[1][0]/x[1][5], x[1][1]/x[1][5], x[1][2]/x[1][5], x[1][3]/x[1][5], x[1][4]/x[1][5]]))
# ratio.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('common_user', [0.5187940159186565, 0.10415885318998207, 0.04371379755802808, 0.2291744801433512, 0.10415885318998207]), ('gamer', [0.2356474564033654, 0.47130494078478957, 0.047382196327754436, 0.11029773066856528, 0.1353676758155253]), ('student', [0.34585368135304295, 0.31998087473819703, 0.057382714851701266, 0.15209433743267614, 0.12468839162438262])]

In [21]:
ratio.coalesce(1).saveAsTextFile('s3://userconfig-6998/ratio_output')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…